In [2]:
import os
import math
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter, find_peaks, tukey
from ssqueezepy import ssq_cwt
import matplotlib.pyplot as plt

In [27]:
def bin_data(data,fs):
  t_seq = np.array(data['timestamp'])
  t0 = t_seq[0]
  num = np.floor((t_seq - t0)/(1000/fs))
  j = 0; i = 1
  new_data = []
  while i<(len(t_seq)-1):
    if num[i]==num[j]:
      i = i + 1
      if i == len(t_seq):
        index = np.arange(j,i)
        mean_x = np.mean(np.array(data['x'])[index])
        mean_y = np.mean(np.array(data['y'])[index])
        mean_z = np.mean(np.array(data['z'])[index])
        new_data.append([t0+1000/fs*num[i],mean_x,mean_y,mean_z])
    else:
      index = np.arange(j,i)
      mean_x = np.mean(np.array(data['x'])[index])
      mean_y = np.mean(np.array(data['y'])[index])
      mean_z = np.mean(np.array(data['z'])[index])
      new_data.append([t0+1000/fs*num[i],mean_x,mean_y,mean_z])
      j = i
      i = i+1
  new_data = np.array(new_data).reshape((len(new_data),4))
  mag = np.sqrt(new_data[:,1]**2+new_data[:,2]**2+new_data[:,3]**2)
  if np.mean(mag)>6:
    mag = mag/9.8
  stamp = (new_data[:,0]-new_data[0,0])/1000
  return stamp,mag

In [4]:
def split_burst(stamp):
  burst_index = []
  j = 0
  if len(stamp)>50:
    for i in range(1,len(stamp)):
      if stamp[i]-stamp[i-1]>1:
        start_index = j
        end_index = i-1
        j = i
        if end_index - start_index>40:
          burst_index.append([start_index,end_index])
    start_index = j
    end_index = len(stamp)-1
    burst_index.append([start_index,end_index])
    if len(burst_index)>1:
      if burst_index[-1]==burst_index[-2]:
        burst_index = burst_index[:-1]
  return burst_index

## bandpass filter
def bandpass_filter(burst, lowcut, highcut, order, fs):
  b, a = butter(order, [lowcut, highcut], btype='bandpass',fs=fs)
  y = lfilter(b, a, burst)
  return y

def adjust_burst(tapered_burst,fs):
  burst = tapered_burst[5*fs:-5*fs]
  if len(burst)%fs>=0.7*fs:
    for i in range(fs-len(burst)%fs):
      burst = np.append(burst,burst[-1])
  elif len(burst)%fs!=0:
    burst = burst[np.arange(len(burst)//fs*fs)]
  return burst

def adjust_out0(out,fs):
  temp = out[0].copy()
  temp = temp[:,5*fs:-5*fs]
  if temp.shape[1]%fs>=0.7*fs:
    for i in range(fs-temp.shape[1]%fs):
      temp = np.append(temp,out[0][:,-1].reshape(-1,1),1)
  elif temp.shape[1]%fs!=0:
    temp = temp[:,np.arange(temp.shape[1]//fs*fs)]
  return np.array(temp)

In [5]:
def get_walk_prop(all_WI):
  total_len = 0
  total_walk = 0
  for i in range(len(all_WI)):
    WI = all_WI[i]
    total_len = total_len + len(WI)
    total_walk = total_walk + sum(WI)
  return total_walk/(total_len+0.001)

In [6]:
## function checks if identified peaks appear consistently in time (T)
## and frequency (delta) domains
def findContinuousDominantPeaks(E,T,delta):
  B = np.zeros((E.shape[0],E.shape[1]))
  for m in range(E.shape[1]-T):
    A = E[:,np.arange(m,m+T)]
    loop = [i for i in np.arange(T)] + [i for i in np.arange(T-2,-1,-1)]
    for t in range(len(loop)):
      s = loop[t]
      pr = np.where(A[:,s]!=0)[0]
      j = 0
      if len(pr)>0:
        for i in range(len(pr)):
          index = np.arange(max(0,pr[i]-delta),min(pr[i]+delta+1,A.shape[0]))
          if s == 0 or s == T-1:
            W = np.transpose(np.array([np.ones(len(index))*pr[i],index],dtype=int))
          else:
            W = np.transpose(np.array([index,np.ones(len(index))*pr[i],index],dtype=int))
          F = np.zeros((W.shape[0],W.shape[1]),dtype=int)
          if s == 0:
            F[:,0] = A[W[:,0],s]
            F[:,1] = A[W[:,1],s+1]
          elif s == T-1:
            F[:,0] = A[W[:,0],s]
            F[:,1] = A[W[:,1],s-1]
          else:
            F[:,0] = A[W[:,0],s-1]
            F[:,1] = A[W[:,1],s]
            F[:,2] = A[W[:,2],s+1]
          G1 = W[np.sum(F[:,np.arange(2)],axis=1)>1,:]
          if s == 0 or s == T-1:
            if G1.shape[0]==0:
              A[W[:,0],s] = 0
            else:
              j = j + 1
          else:
            G2 = W[np.sum(F[:,np.arange(1,3)],axis=1)>1,:]
            if G1.shape[0]==0 or G2.shape[0]==0:
              A[W[:,1],s] = 0
            else:
              j = j + 1
      if j == 0:
        A = np.zeros((A.shape[0],A.shape[1]))
        break
    B[:,np.arange(m,m+T)] = np.maximum(B[:,np.arange(m,m+T)],A)
  return B

In [7]:
def morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta):
  w = tukey(len(burst), alpha=0.02, sym=True) 
  tapered_burst = np.concatenate((np.zeros(5*fs),(burst-1)*w,np.zeros(5*fs)))
  wavelet = ('gmw', {'beta': 90,'gamma':3})
  try:
    out = ssq_cwt(tapered_burst,wavelet,fs=10)
  except:
    tapered_burst = tapered_burst[:-1]
    out = ssq_cwt(tapered_burst,wavelet,fs=10)
  temp = adjust_burst(tapered_burst,fs)
  dur = len(temp)/fs
  filtered_burst = bandpass_filter(temp, lowcut, highcut, order, fs)
  x = filtered_burst.reshape(fs,-1,order="F") 
  pp = np.array([max(x[:,i])-min(x[:,i]) for i in range(x.shape[1])])
  valid = np.ones(len(pp),dtype=bool)
  valid[pp<min_amp]=False 
  index = (out[2]>0.6)*(out[2]<4.8)
  freqs = out[2][index]
  coefs = adjust_out0(out,fs)[index,:]
  coefs2 = np.abs(coefs**2)
  D = np.zeros((coefs2.shape[0],int(coefs2.shape[1]/fs)))
  loc_min = np.argmin(abs(freqs-step_freq[0]))
  loc_max = np.argmin(abs(freqs-step_freq[1]))
  for i in range(int(coefs2.shape[1]/fs)):
    # segment measurement into one-second non-overlapping windows
    xStart  = i*fs
    xFinish = (i+1)*fs
    # identify peaks and their l ocation in each window
    window = np.sum(coefs2[:,np.arange(xStart,xFinish)],axis=1)
    locs,_ = find_peaks(window)
    pks = window[locs]
    I = np.argsort(-pks)
    locs = locs[I]
    pks = pks[I]
    index_in_range = []
    for j in range(len(locs)):
      if locs[j]>=loc_min and locs[j]<=loc_max:
        index_in_range.append(j)
      if len(index_in_range)>=1:
        break
    x = np.zeros(coefs2.shape[0])
    if len(index_in_range)>0:
      if locs[0]>loc_max:
        if pks[0]/pks[index_in_range[0]]<alpha:
          x[locs[index_in_range[0]]]=1
      elif locs[0]<loc_min:
        if pks[0]/pks[index_in_range[0]]<beta:
          x[locs[index_in_range[0]]]=1
      else:
        x[locs[index_in_range[0]]]=1
    D[:,i]=x
  E = np.zeros((D.shape[0],D.shape[1]))
  E[:,valid] = D[:,valid]
  B = findContinuousDominantPeaks(E,T,delta)
  WI = np.zeros(E.shape[1])
  e = np.sum(B,axis=0)
  WI[e>0]=1
  cad = np.zeros(E.shape[1])
  for i in range(len(cad)):
    ind_freqs = np.where(B[:,i]>0)[0]
    if len(ind_freqs)>0:
      cad[i] = freqs[ind_freqs[0]]
  return int(round(sum(cad),0)),int(dur),WI

In [9]:
fs = 10
lowcut = 0.6
highcut = 4.6
order = 3
min_amp = 0.2
step_freq = [1.2,2.3]
alpha = 4
beta = 2
T = 5
delta = 3

In [30]:
## test on a dataset
data = pd.read_csv("C:/Users/glius/Google Drive/Thesis/paper 3/walk_dataset/1/acc_walking_thigh.csv")
data.columns = ["id","timestamp","x","y","z"]
data.head(5)

,id,timestamp,x,y,z
0,1,1435993159419,0.055665,9.621099,-0.011372
1,2,1435993159437,0.073622,9.554062,-0.077812
2,3,1435993159456,0.077812,9.537901,-0.155025
3,4,1435993159475,0.113725,9.567230,-0.159214
4,5,1435993159499,0.116717,9.599552,-0.104746


In [31]:
stamp,mag = bin_data(data,fs)
burst_index = split_burst(stamp)
burst_index

[[0, 906],
 [907, 960],
 [961, 1014],
 [1015, 2892],
 [2893, 2986],
 [2987, 3120],
 [3121, 3197],
 [3198, 4875],
 [4876, 5007],
 [5008, 5223],
 [5224, 5341],
 [5342, 6214]]

In [32]:
i = 5
burst = mag[np.arange(burst_index[i][0],burst_index[i][1]+1)]
morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta)

(19, 13, array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]))

In [33]:
i = 11
burst = mag[np.arange(burst_index[i][0],burst_index[i][1]+1)]
morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta)

(136,
 87,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0.]))

In [35]:
## test on all individuals and locations
final_steps = np.zeros((15,7))
final_props = np.zeros((15,7))
for i in range(15):
  path = "C:/Users/glius/Google Drive/Thesis/paper 3/walk_dataset/"+str(i+1)+"/"
  for file in os.listdir(path):
    if "acc" in file:
      data = pd.read_csv(path+file)
      data.columns = ["id","timestamp","x","y","z"]
      stamp,mag = bin_data(data,fs)
      burst_index = split_burst(stamp)
      total_step = 0
      all_WI = []
      for j in range(len(burst_index)):
        print(i,file,j)
        burst = mag[np.arange(burst_index[j][0],burst_index[j][1]+1)]
        step,dur,WI = morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta)
        total_step = total_step + step
        all_WI.append(WI)
      total_prop = get_walk_prop(all_WI)
      if "chest" in file:
        final_steps[i,0] = total_step
        final_props[i,0] = total_prop
      elif "forearm" in file:
        final_steps[i,1] = total_step
        final_props[i,1] = total_prop
      elif "head" in file:
        final_steps[i,2] = total_step
        final_props[i,2] = total_prop
      elif "shin" in file:
        final_steps[i,3] = total_step
        final_props[i,3] = total_prop
      elif "thigh" in file:
        final_steps[i,4] = total_step
        final_props[i,4] = total_prop
      elif "upperarm" in file:
        final_steps[i,5] = total_step
        final_props[i,5] = total_prop
      else:    ## waist
        final_steps[i,6] = total_step
        final_props[i,6] = total_prop
np.savetxt("C:/Users/glius/Downloads/walk_step_tapered.csv", final_steps, delimiter=",")
np.savetxt("C:/Users/glius/Downloads/walk_prop_tapered.csv", final_props, delimiter=",")

0 acc_walking_chest.csv 0
0 acc_walking_chest.csv 1
0 acc_walking_chest.csv 2
0 acc_walking_chest.csv 3
0 acc_walking_chest.csv 4
0 acc_walking_chest.csv 5
0 acc_walking_chest.csv 6
0 acc_walking_chest.csv 7
0 acc_walking_chest.csv 8
0 acc_walking_chest.csv 9
0 acc_walking_chest.csv 10
0 acc_walking_chest.csv 11
0 acc_walking_chest.csv 12
0 acc_walking_chest.csv 13
0 acc_walking_forearm.csv 0
0 acc_walking_forearm.csv 1
0 acc_walking_forearm.csv 2
0 acc_walking_forearm.csv 3
0 acc_walking_forearm.csv 4
0 acc_walking_forearm.csv 5
0 acc_walking_forearm.csv 6
0 acc_walking_forearm.csv 7
0 acc_walking_head.csv 0
0 acc_walking_head.csv 1
0 acc_walking_head.csv 2
0 acc_walking_head.csv 3
0 acc_walking_head.csv 4
0 acc_walking_head.csv 5
0 acc_walking_head.csv 6
0 acc_walking_head.csv 7
0 acc_walking_head.csv 8
0 acc_walking_head.csv 9
0 acc_walking_head.csv 10
0 acc_walking_head.csv 11
0 acc_walking_head.csv 12
0 acc_walking_shin.csv 0
0 acc_walking_shin.csv 1
0 acc_walking_shin.csv 2
0 acc

C:\Users\glius\anaconda3\lib\site-packages\ssqueezepy\ssqueezing.py:235: RuntimeWarning: invalid value encountered in double_scalars
  a = (fmin**tmax / fmax**tmin) ** (1/(tmax - tmin))


0 acc_walking_thigh.csv 8
0 acc_walking_thigh.csv 9
0 acc_walking_thigh.csv 10
0 acc_walking_thigh.csv 11
0 acc_walking_upperarm.csv 0
0 acc_walking_upperarm.csv 1
0 acc_walking_upperarm.csv 2
0 acc_walking_upperarm.csv 3
0 acc_walking_upperarm.csv 4
0 acc_walking_waist.csv 0
0 acc_walking_waist.csv 1
0 acc_walking_waist.csv 2
0 acc_walking_waist.csv 3
1 acc_walking_chest.csv 0
1 acc_walking_chest.csv 1
1 acc_walking_chest.csv 2
1 acc_walking_chest.csv 3
1 acc_walking_chest.csv 4
1 acc_walking_chest.csv 5
1 acc_walking_chest.csv 6
1 acc_walking_chest.csv 7
1 acc_walking_chest.csv 8
1 acc_walking_chest.csv 9
1 acc_walking_chest.csv 10
1 acc_walking_chest.csv 11
1 acc_walking_forearm.csv 0
1 acc_walking_forearm.csv 1
1 acc_walking_forearm.csv 2
1 acc_walking_forearm.csv 3
1 acc_walking_forearm.csv 4
1 acc_walking_forearm.csv 5
1 acc_walking_head.csv 0
1 acc_walking_head.csv 1
1 acc_walking_head.csv 2
1 acc_walking_head.csv 3
1 acc_walking_head.csv 4
1 acc_walking_head.csv 5
1 acc_walking

4 acc_walking_upperarm.csv 2
4 acc_walking_upperarm.csv 3
4 acc_walking_upperarm.csv 4
4 acc_walking_upperarm.csv 5
4 acc_walking_upperarm.csv 6
4 acc_walking_upperarm.csv 7
4 acc_walking_upperarm.csv 8
4 acc_walking_upperarm.csv 9
4 acc_walking_upperarm.csv 10
4 acc_walking_upperarm.csv 11
4 acc_walking_upperarm.csv 12
4 acc_walking_waist.csv 0
4 acc_walking_waist.csv 1
4 acc_walking_waist.csv 2
4 acc_walking_waist.csv 3
4 acc_walking_waist.csv 4
4 acc_walking_waist.csv 5
4 acc_walking_waist.csv 6
4 acc_walking_waist.csv 7
4 acc_walking_waist.csv 8
4 acc_walking_waist.csv 9
4 acc_walking_waist.csv 10
4 acc_walking_waist.csv 11
4 acc_walking_waist.csv 12
5 acc_walking_chest.csv 0
5 acc_walking_chest.csv 1
5 acc_walking_chest.csv 2
5 acc_walking_chest.csv 3
5 acc_walking_chest.csv 4
5 acc_walking_chest.csv 5
5 acc_walking_chest.csv 6
5 acc_walking_chest.csv 7
5 acc_walking_chest.csv 8
5 acc_walking_chest.csv 9
5 acc_walking_chest.csv 10
5 acc_walking_chest.csv 11
5 acc_walking_chest.csv

8 acc_walking_shin.csv 0
8 acc_walking_shin.csv 1
8 acc_walking_shin.csv 2
8 acc_walking_shin.csv 3
8 acc_walking_shin.csv 4
8 acc_walking_shin.csv 5
8 acc_walking_shin.csv 6
8 acc_walking_shin.csv 7
8 acc_walking_shin.csv 8
8 acc_walking_shin.csv 9
8 acc_walking_shin.csv 10
8 acc_walking_shin.csv 11
8 acc_walking_shin.csv 12
8 acc_walking_thigh.csv 0
8 acc_walking_thigh.csv 1
8 acc_walking_thigh.csv 2
8 acc_walking_thigh.csv 3
8 acc_walking_thigh.csv 4
8 acc_walking_thigh.csv 5
8 acc_walking_thigh.csv 6
8 acc_walking_thigh.csv 7
8 acc_walking_thigh.csv 8
8 acc_walking_thigh.csv 9
8 acc_walking_thigh.csv 10
8 acc_walking_thigh.csv 11
8 acc_walking_thigh.csv 12
8 acc_walking_upperarm.csv 0
8 acc_walking_upperarm.csv 1
8 acc_walking_upperarm.csv 2
8 acc_walking_upperarm.csv 3
8 acc_walking_upperarm.csv 4
8 acc_walking_upperarm.csv 5
8 acc_walking_upperarm.csv 6
8 acc_walking_upperarm.csv 7
8 acc_walking_upperarm.csv 8
8 acc_walking_upperarm.csv 9
8 acc_walking_upperarm.csv 10
8 acc_walki

12 acc_walking_2_chest.csv 0
12 acc_walking_2_chest.csv 1
12 acc_walking_2_chest.csv 2
12 acc_walking_2_chest.csv 3
12 acc_walking_2_chest.csv 4
12 acc_walking_2_chest.csv 5
12 acc_walking_2_chest.csv 6
12 acc_walking_2_chest.csv 7
12 acc_walking_2_chest.csv 8
12 acc_walking_2_chest.csv 9
12 acc_walking_2_chest.csv 10
12 acc_walking_2_forearm.csv 0
12 acc_walking_2_forearm.csv 1
12 acc_walking_2_forearm.csv 2
12 acc_walking_2_forearm.csv 3
12 acc_walking_2_forearm.csv 4
12 acc_walking_2_forearm.csv 5
12 acc_walking_2_forearm.csv 6
12 acc_walking_2_head.csv 0
12 acc_walking_2_head.csv 1
12 acc_walking_2_head.csv 2
12 acc_walking_2_head.csv 3
12 acc_walking_2_head.csv 4
12 acc_walking_2_head.csv 5
12 acc_walking_2_head.csv 6
12 acc_walking_2_head.csv 7
12 acc_walking_2_head.csv 8
12 acc_walking_2_head.csv 9
12 acc_walking_2_head.csv 10
12 acc_walking_2_shin.csv 0
12 acc_walking_2_shin.csv 1
12 acc_walking_2_shin.csv 2
12 acc_walking_2_shin.csv 3
12 acc_walking_2_shin.csv 4
12 acc_walking

In [20]:
### test on another dataset
data = pd.read_csv("C:/Users/glius/Downloads/activity-recognition-dataset-shoaib/Activity_Recognition_DataSet/Pocket.csv")
colnames = np.array(data.columns)
colnames[0:4] = ["timestamp","x","y","z"]
data.columns = colnames
data.head(5)

,timestamp,x,y,z,Gx,Gy,Gz,Mx,My,Mz,Activity_Label
0,1.360000e+12,0.231546,-9.329938,-3.023717,0.275195,-0.111788,0.019242,-19.680000,29.640000,-6.60,Downstairs
1,1.360000e+12,1.048767,-8.853226,-1.225831,0.801150,1.128879,0.011912,-19.619999,29.400000,-6.72,Downstairs
2,1.360000e+12,0.939804,-9.507003,-2.043052,0.807258,1.709201,-0.032376,-19.680000,29.340000,-6.96,Downstairs
3,1.360000e+12,0.640156,-9.833891,-2.152015,0.602008,2.062892,0.003665,-19.859999,29.220000,-7.14,Downstairs
4,1.360000e+12,0.027241,-9.248216,-1.321174,0.629802,2.191174,0.159741,-19.680000,29.039999,-7.98,Downstairs


In [21]:
data = data[data["Activity_Label"]=="Walking"]
data["timestamp"] = data["timestamp"] + np.array([20*i for i in range(data.shape[0])])
data.head(5)

,timestamp,x,y,z,Gx,Gy,Gz,Mx,My,Mz,Activity_Label
130706,1.360000e+12,1.416516,-9.779409,-1.784265,0.311541,0.379958,0.616058,3.06,26.580000,26.279999,Walking
130707,1.360000e+12,1.389275,-10.351464,-1.620821,0.326813,0.469450,0.618806,3.18,26.939999,25.920000,Walking
130708,1.360000e+12,1.443757,-10.188020,-1.811506,0.367741,0.521373,0.598037,3.36,27.119999,25.680000,Walking
130709,1.360000e+12,1.893228,-10.869038,-2.669588,0.374766,0.558942,0.574824,3.30,27.359999,25.500000,Walking
130710,1.360000e+12,3.282504,-11.618156,-4.181447,0.243430,0.532980,0.467006,3.60,27.660000,25.140000,Walking


In [35]:
fs = 10
lowcut = 0.6
highcut = 4.6
order = 3
min_amp = 0.2
step_freq = [1.2,2.3]
alpha = 4
beta = 2
T = 5
delta = 3
stamp,mag = bin_data(data,fs)
burst_index = split_burst(stamp)
burst_index

[[0, 6249]]

In [37]:
i = 0
burst = mag[np.arange(burst_index[i][0],burst_index[i][1]+1)]
step,dur,WI = morse_step_count_burst(burst,fs,lowcut,highcut,order,min_amp,step_freq,alpha,beta,T,delta)

In [38]:
step

1042

In [39]:
sum(WI)/len(WI)

0.8512